<a href="https://colab.research.google.com/github/mn300/R_for_DS/blob/main/pygge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pygge - a collection of useful functions to make learning Python for satellite image analysis easy

Developed by Heiko Balzter at the University of Leicester

In [ ]:
#import required libraries


# the following pip install commands must be added to the main program
'''
!pip install rasterio
!pip install sentinelsat
!pip install geopandas
!pip install rasterstats
'''

from collections import OrderedDict
import csv
import ee
import fnmatch
import geopandas as gpd
import io
import math
from math import floor, ceil
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import ogr
import os
from os import listdir
from os.path import isfile, isdir, join
import pandas as pd
import pickle
from pyproj import Proj
from pprint import pprint
import random
import rasterio
from rasterio.windows import Window
from rasterio import features, plot
from rasterio.plot import show_hist, reshape_as_raster, reshape_as_image
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats
import requests
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt, geojson
from scipy import optimize
import skimage.io as io
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import joblib
#from sklearn.externals import joblib # depracated
import shutil
import sys
import webbrowser
import zipfile

In [ ]:
def scale_to_uint8(x, percentiles=[0,100]):
    '''
    Scales an array to the range from 0-255 and converts the data type to uint8.
    NaN values will be ignored.

    Args:
      x = input array
      percentiles = list of length 2 of percentiles for trimming the histogram (0-100)

    Returns:
      Scaled array of uint8 data type
    '''

    x = np.float32(x)
    amin = np.nanpercentile(x, percentiles[0])
    amax = np.nanpercentile(x, percentiles[1])
    anewmin = 0.0
    anewmax = 255.0

    # apply percentile thresholds
    x[x<amin] = amin
    x[x>amax] = amax

    # scale values
    xscaled = (x - amin) / (amax - amin) * (anewmax - anewmin) + anewmin

    return(xscaled.astype(np.uint8))

In [ ]:
def longlat2window(lon, lat, dataset):
    """
    Converts a tuple of longitude and latitude coordinates into pixel locations
    (rows, columns) in a raster and checks whether the coordinates are within
    the raster extent.
    If coordinates are outside the raster extent, the pixel locations will be
    changed to make sure they stay within the raster extent.

    Args:
        lon (tuple): Tuple of min and max lon
        lat (tuple): Tuple of min and max lat
        dataset: Rasterio dataset

    Returns:
        rasterio.windows.Window
    """

    # get coordinate referencing system from the raster file
    p = Proj(dataset.crs)
    # get geotransform
    t = dataset.transform

    # convert longitude and latitude from map coordinates to raster locations (pixel/line position)
    xmin, ymin = p(lon[0], lat[0])
    xmax, ymax = p(lon[1], lat[1])
    col_min, row_min = ~t * (xmin, ymin) # uses the geotransform to convert the coordinates
    col_max, row_max = ~t * (xmax, ymax)

    # check whether the window coordinates are within the raster file bounds
    # and adjust them to fit within the raster file if they are outside it
    y0 = floor(row_max)
    if y0 < 0:
      y0 = 0
    if y0 > dataset.height:
      y0 = dataset.height
    y1 = ceil(row_min)
    if y1 < 0:
      y1 = 0
    if y1 > dataset.height:
      y1 = dataset.height
    x0 = floor(col_min)
    if x0 < 0:
      x0 = 0
    if x0 > dataset.width:
      x0 = dataset.width
    x1 = ceil(col_max)
    if x1 < 0:
      x1 = 0
    if x1 > dataset.width:
      x1 = dataset.width

    return Window.from_slices(rows=(y0, y1), cols=(x0, x1))

In [ ]:
def copy_and_overwrite(from_path, to_path, delete_target_dir=True):
  '''
  Copies the content of the directory from_path to to_path.
  Deletes the to_path directory before copying if that option is selected.

  Modified from: https://stackoverflow.com/questions/12683834/how-to-copy-directory-recursively-in-python-and-overwrite-all

  Args:
    from_path = string of source directory path to be copied
    to_path = string of destination directory path to be copied to
    delete_target_dir = True or False. If True, the destination directory will
      be deleted before copying.
  '''

  if os.path.exists(to_path):
    if delete_target_dir:
      shutil.rmtree(to_path)
      shutil.copytree(from_path, to_path)
    else:
      print("Error: Target directory exists and delete_target_dir is set to False.")
  else:
    shutil.copytree(from_path, to_path)
  return()


In [ ]:
# To make efficient use of Google Earth Engine from Python, we want to define some useful helper functions from https://climada-python.readthedocs.io/en/stable/tutorial/climada_util_earth_engine.html
# Functions modified from climada.util.earth_engine module

def obtain_image_landsat_composite(collection, time_range, area):
    """ Selection of Landsat cloud-free composites in the Earth Engine library
    See also: https://developers.google.com/earth-engine/landsat

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_composite (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_composite = ee.Algorithms.Landsat.simpleComposite(image_area, 75, 3)
    return image_composite

def obtain_image_median(collection, time_range, area):
    """ Selection of median from a collection of images in the Earth Engine library
    See also: https://developers.google.com/earth-engine/reducers_image_collection

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_median (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_median = image_area.median()
    return image_median

'''
The function below has been modified to accept the cloud cover threshold as an input
'''
def obtain_image_sentinel(collection, time_range, area, clouds):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """

    #First, method to remove cloud from the image
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clouds)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.median()
    return sentinel_median

def obtain_image_collection_sentinel(collection, time_range, area, clouds):
    """ Selection of a cloud-free image collection in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        s2collect (image collection)
     """

    #First, method to remove cloud from the image
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    s2collect = (ee.ImageCollection(collection).
                filterBounds(area).
                filterDate(time_range[0], time_range[1]).
                filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clouds)).
                map(maskclouds))

    return s2collect

def get_region(geom):
    """Get the region of a given geometry, needed for exporting tasks.

    Parameters:
        geom (ee.Geometry, ee.Feature, ee.Image): region of interest

    Returns:
        region (list)
    """
    if isinstance(geom, ee.Geometry):
        region = geom.getInfo()["coordinates"]
    elif isinstance(geom, ee.Feature, ee.Image):
        region = geom.geometry().getInfo()["coordinates"]
    elif isinstance(geom, list):
        condition = all([isinstance(item) == list for item in geom])
        if condition:
            region = geom
    return region

def get_url(name, image, scale, region, filePerBand=False):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    Parameters:
        name -  a base name to use when constructing filenames.
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest
        filePerBand - whether to produce a different GeoTIFF per band (boolean).
            Defaults to true. If false, a single GeoTIFF is produced and all
            band-level transformations will be ignored.

    Returns:
        path (str)


    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Args:
        params: An object containing visualization options with the following
          possible values:
        name -  a base name to use when constructing filenames.
        bands -  a description of the bands to download. Must be an array of
            dictionaries, each with the following keys:
          id -  the name of the band, a string, required.
          crs -  an optional CRS string defining the band projection.
          crs_transform -  an optional array of 6 numbers specifying an affine
              transform from the specified CRS, in the order: xScale, yShearing,
              xShearing, yScale, xTranslation and yTranslation.
          dimensions -  an optional array of two integers defining the width and
              height to which the band is cropped.
          scale -  an optional number, specifying the scale in meters of the
                 band; ignored if crs and crs_transform is specified.
        crs -  a default CRS string to use for any bands that do not explicitly
            specify one.
        crs_transform -  a default affine transform to use for any bands that do
            not specify one, of the same format as the crs_transform of bands.
        dimensions -  default image cropping dimensions to use for any bands
            that do not specify them.
        scale -  a default scale to use for any bands that do not specify one;
            ignored if crs and crs_transform is specified.
        region -  a polygon specifying a region to download; ignored if crs
            and crs_transform is specified.
        filePerBand - whether to produce a different GeoTIFF per band (boolean).
            Defaults to true. If false, a single GeoTIFF is produced and all
            band-level transformations will be ignored.
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region),
        'filePerBand': (filePerBand)
        })

    webbrowser.open_new_tab(path)
    return path

In [ ]:
# search Copernicus Open Data Hub
# search Copernicus Open Data Hub
def search_Cop_Hub(api, username, password, shapefile, datefrom, dateto, **kwargs):
  '''
  Carries out a search for Sentinel data on the Copernicus Open Data Hub.

  Args:
    api = string pointing to the API of the Data Hub
    username = username for logging into the Copernicus Data Hub
    password = password for logging into the Copernicus Data Hub
    shapefile = string containing the path and file name of a shapefile for the search area
    datefrom = string with the start date of the search, in the form YYYYMMDD
    dateto = string with the end date of the search, in the same form
    kwargs = additional keyword arguments that match the properties of the Sentinel mission, e.g.
      for Sentinel-1:
        platformname = name of the Sentinel platform, i.e. 'Sentinel-1'
        producttype = product type, e.g. 'SLC' or 'GRD'
      for Sentinel-2:
        platformname = name of the Sentinel platform, i.e. 'Sentinel-2'
        processinglevel = processing level of the data, e.g. 'Level-2A'
        clouds = string with maximum cloud cover in the form '[0 TO 50]'

  Returns an api object and an ordered dictionary of the search results


  Notes:
  Call this function like this:
  # test functionality of S-1 query
  kwargs = {
            'platformname' : 'Sentinel-1',
            'producttype' : 'GRD'
          }

  # test functionality of S-2 query
  kwargs = {
            'platformname' : 'Sentinel-2',
            'processinglevel' : 'Level-2A',
            'clouds' : '[0 TO 100]'
          }

  # do the search
  api, products = search_Cop_Hub(api='https://apihub.copernicus.eu/apihub/',
                                     username=username,
                                     password=password,
                                     shapefile=shapefile,
                                     datefrom=datefrom,
                                     dateto=dateto,
                                     **kwargs)


  '''

  API = SentinelAPI(username, password, api)

  # Get the shapefile layer's extent
  extent, SpatialRef, epsg = get_shp_extent(shapefile)

  # set query parameters and search the Copernicus Open Data Hub
  kwargs['area'] = bbox(extent)
  kwargs['date'] = (datefrom, dateto)
  try:
    kwargs = {'cloudcoverpercentage' if k == 'clouds' else k:v for k,v in kwargs.items()}
  except NameError:
    pass

  print("Copernicus Open Access Hub search parameters:")
  for key, value in kwargs.items():
    print(key, " = ", value)

  # search the Sentinel data hub API
  products = API.query(**kwargs)

  # the search returns an ordered dictionary object of the image products
  return API, products


def search_Cop_Hub_old(api, username, password, shapefile, datefrom, dateto,
                   platformname, processinglevel, clouds):
  '''
  Carries out a search for Sentinel data on the Copernicus Open Data Hub.

  Args:
    api = string pointing to the API of the Data Hub
    username = username for logging into the Copernicus Data Hub
    password = password for logging into the Copernicus Data Hub
    shapefile = string containing the path and file name of a shapefile for the search area
    datefrom = string with the start date of the search, in the form YYYYMMDD
    dateto = string with the end date of the search, in the same form
    platformname = name of the Sentinel platform, e.g. 'Sentinel-2'
    processinglevel = processing level of the data, e.g. 'Level-2A'
    clouds = string with maximum cloud cover in the form '[0 TO 50]'

  Returns an api object and an ordered dictionary of the search results
  '''

  print("This is the version from the practicals 2021-22 - now depracated")

  API = SentinelAPI(username, password, api)

  # Get the shapefile layer's extent
  extent, SpatialRef, epsg = get_shp_extent(shapefile)

  #print("Extent of the shapefile: \n", extent)
  #print("Spatial referencing information of the shapefile:\n", SpatialRef)
  #print("EPSG code of the map projection: ", epsg)

  # set query parameters and search the Copernicus Open Data Hub
  kwargs = {
          'area': bbox(extent),
          'date': (datefrom, dateto),
          'platformname': platformname,
          'processinglevel': processinglevel,
          'cloudcoverpercentage': clouds
          }

  # search the Sentinel data hub API
  products = API.query(**kwargs)

  # the search returns an ordered dictionary object of the image products
  return API, products

In [ ]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Get acquisition date from the id column
    df['date'] = df['id']

    for i in range(len(df['date'])):
        x0 = df['date'].values[i]
        x1 = x0.split("_")[0]
        x2 = x1.split("T")[0]
        # print(i, df['date'][i], x2)
        df['date'].values[i] = x2

    # Keep the columns of interest.
    df = df[['date',  'id', *list_of_bands]]

    # drop all rows with NaN values in either of the bands
    df = df.dropna(0)

    return df

In [ ]:
# Split string of format YYYYMMDD into three parts Y, M, D
def split_YYYYMMDD(word):
  # split into individual characters
  chars = [char for char in word]
  Y = np.int64(''.join(chars[0:4]))
  M = np.int64(''.join(chars[4:6]))
  D = np.int64(''.join(chars[6:8]))
  return Y, M, D

def julian_date(y, m, d):
  # convert year, month, day into Julian date
  # after https://quasar.as.utexas.edu/BillInfo/JulianDatesG.html
  # if the month is January or February, subtract 1 from the year to get a new Y,
  #   and add 12 to the month to get a new M.
  #   (Thus, we are thinking of January and February as being the 13th and 14th month
  #    of the previous year).
  if m == 1 or m == 2:
    m = m + 12
    y = y - 1

  # dropping the fractional part of all results of all multiplications and divisions, let
  a = y / 100
  b = a / 4
  c = 2 - a + b
  e = 365.25 * (y + 4716)
  f = 30.6001 * (m + 1)

  return math.floor(c + d + e + f - 1524.5)

In [ ]:
def easy_clip(rasterfile, clippedfile, extent):
  '''
  This function clips an input raster file to a geographic map extent.
  rasterio offers an option called 'window' to load a subset of a raster file

  Args:
    rasterfile = directory path and file name of the input raster file
    clippedfile = directory path and file name of the clipped output raster file
    extent = extent object to define the clipping area
  '''

  # open the source file
  with rasterio.open(rasterfile, 'r') as src:

    # convert the shapefile extent to a rasterio window object
    window = longlat2window((extent[0], extent[1]), (extent[2], extent[3]), src)
    print("Window coordinates: ", window)

    # read all bands but only for the window extent
    arr = src.read(window=window, out_shape=(src.count, window.height, window.width))
    print("Window array size: ", arr.shape)

    # get the data type
    dt = arr.dtype

    # open the destination file
    # copy metadata from source file
    # BUT we must change the geotransform to the window with the update below
    # https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html
    kwargs = src.meta.copy()
    kwargs.update({'height': window.height,
                    'width': window.width,
                    'transform': rasterio.windows.transform(window, src.transform),
                    'driver': 'Gtiff',
                    'count': src.count,
                    'crs': src.crs,
                    'dtype': dt
                    })

    with rasterio.open(clippedfile, 'w', **kwargs) as dst:
      dst.write(arr)

      # close the destination file
      dst.close()

    # close the sourcefile
    src.close()
  return()

In [ ]:
def get_filenames(path, filepattern, dirpattern):
  '''
  Finds all file names in a directory for which the file name matches a certain string pattern,
    and the directory name matches a different string pattern.
  Args:
    path = string indicating the path to a directory in which the search will be done
    filepattern = string of the file name pattern to search for
    dirpattern = string of the directory name pattern to search for

  Returns:
    a list of all found files with the full path directory
  '''

  filelist = []

  for root, dirs, files in os.walk(path, topdown=True):
    dirs[:] = [d for d in dirs]
    for f in files:
      if filepattern in f and dirpattern in root:
        thisfile = os.path.join(root, f)
        filelist.append(thisfile)

  return(sorted(filelist))

In [ ]:
def bbox(extent):
  '''
  Converts coordinates in the form of an extent object to a bounding box object.

  Args:
    extent = extent object to be converted to a bounding box object

  Returns:
    a polygon bounding box object
  '''

  # We need to define a helper function that creates a simply bounding box polygon from the
  #   extent of our shapefile in the right format for the Data Hub API.
  # Create a Polygon from the extent tuple
  box = ogr.Geometry(ogr.wkbLinearRing)
  box.AddPoint(extent[0],extent[2])
  box.AddPoint(extent[1], extent[2])
  box.AddPoint(extent[1], extent[3])
  box.AddPoint(extent[0], extent[3])
  box.AddPoint(extent[0],extent[2])
  poly = ogr.Geometry(ogr.wkbPolygon)
  poly.AddGeometry(box)
  return poly


In [ ]:
def get_shp_extent(shapefile):
  '''
  Get the extent of the first layer, the CRS and the EPSG code from a shapefile

  Args:
    shapefile = path and filename of the shapefile *.shp

  Returns:
    extent of the shapefile
    coordinate referencing system of the shapefile
    EPSG code of the shapefile
  '''

  driver = ogr.GetDriverByName("ESRI Shapefile")
  ds = driver.Open(shapefile, 0)

  # get extent
  lyr = ds.GetLayer()
  extent = lyr.GetExtent()

  # get projection information
  SpatialRef = lyr.GetSpatialRef()

  # get EPSG code of the CRS
  EPSG = SpatialRef.GetAttrValue("AUTHORITY", 1)

  # close file
  ds = None

  return(extent, SpatialRef, EPSG)

In [ ]:
def easy_warp(rasterfile, warpfile, EPSG, res=rasterio.warp.Resampling.bilinear, skip=False, xres=None, yres=None):
  '''
  Warp a rasterfile to the coordinate reference system defined by the EPSG code
    and resample it to a new x and y pixel size if specified.
  Uses the RasterIO warp function.

  Args:
    rasterfile = string with directory path and filename of the raster file
    warpfile = string with directory path and filename of the warped destination raster file
               (will be created if it does not exist)
    EPSG = integer with the EPSG code of the destination coordinate referencing system
    res = any resampling option from rasterio.warp.Resampling
    skip = True or False. If True, existing warp files will be skipped.
    xres (optional)= output pixel resolution in x direction
    yres (optional)= output pixel resolution in y direction
  '''
  # check whether the warp file already exists and skip if it does
  if not os.path.exists(warpfile):
    print("Creating warped file:" + warpfile)

    dst_crs = 'EPSG:'+str(EPSG) # destination coordinate referencing system

    with rasterio.open(rasterfile) as src:
      transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
      kwargs = src.meta.copy()
      kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
      })

      # resample data to target shape if output pixel size is specified

      # get input raster resolution in x and y
      x_input_resolution, y_input_resolution = spatial_resolution(src)

      # scale in x direction
      if xres is not None:
        scale_factor_x = xres / x_input_resolution
      else:
        scale_factor_x = 1.0

      # scale in y direction
      if yres is not None:
        scale_factor_y = yres / y_input_resolution
      else:
        scale_factor_y = 1.0

      data = src.read(
          out_shape=(
              src.count,
              int(src.height * scale_factor_y),
              int(src.width * scale_factor_x)
          ),
          resampling=Resampling.bilinear
      )

      # scale image transform
      dst_transform = src.transform * src.transform.scale(
          (src.width / data.shape[-1]),
          (src.height / data.shape[-2])
      )

      with rasterio.open(warpfile, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
          reproject(
            source=rasterio.band(src, i),
            destination=rasterio.band(dst, i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=res)

  else:
    print(warpfile + " already exists. Skipping this command.")

In [ ]:
def spatial_resolution(raster):
    """extracts the XY Pixel Size"""
    t = raster.transform
    x = t[0]
    y =-t[4]
    return x, y

In [ ]:
def convert_to_dtype(rasterfile, outfile, out_dtype=np.uint8, scale=True,
                     percentiles=[0,100], maximum=np.nan):
  '''
  Convert a rasterfile to the data type defined in out_dtype.

  Args:
    rasterfile = string with directory path and filename of the raster file
    outfile = string with directory path and filename of the output raster file
    out_dtype = output raster data type, default = np.uint8
    scale = if True, values are scaled to 0-255
    percentiles = values will be capped within the percentile range. Default is
                  to use all values
    maximum = an optional maximum value. If this is not np.nan, then the pixel
              values will be capped at the maximum value given here instead of
              the percentiles
  '''

  def scale_to_uint8(x, percentiles=[0,100], maximum=np.nan):
    '''
    Scales an array to the range from 0-255 and converts the data type to uint8.
    NaN values will be ignored.

    Args:
      x = input array
      percentiles = list of length 2 of percentiles for trimming the histogram (0-100)
      maximum = an optional maximum value. If this is not np.nan, then the pixel
                values will be capped at the maximum value given here instead of
                the percentiles

    Returns:
      Scaled array of uint8 data type
    '''

    x = np.float32(x)

    anewmin = 0.0
    anewmax = 255.0

    if np.isnan(maximum):
      amin = np.nanpercentile(x, percentiles[0])
      amax = np.nanpercentile(x, percentiles[1])
    else:
      amin = 0
      amax = maximum

    print('Scaling from ['+str(amin)+','+str(amax)+'] to [0, 255]')

    # apply percentile thresholds or maximum threshold
    x[x<amin] = amin
    x[x>amax] = amax

    # scale values
    xscaled = (x - amin) / (amax - amin) * (anewmax - anewmin) + anewmin

    return(xscaled.astype(np.uint8))

  with rasterio.open(rasterfile) as src:
    kwargs = src.meta.copy()
    kwargs.update({
      'dtype': out_dtype
    })

    # save the uint8 raster file
    newfile = rasterio.open(outfile, 'w', driver=src.driver,
                            width=src.width, height=src.height,
                            count=src.count, crs=src.crs,
                            transform=src.transform,
                            dtype=out_dtype)

    for i in range(1, src.count + 1):
      band = np.array(src.read(i))

      if (out_dtype == np.uint8 and ((band.min() < 0) or (band.max() > 255))) or scale:
        band = scale_to_uint8(band, percentiles, maximum)

      newfile.write(band.astype(out_dtype), i)

    newfile.close()
    src.close()

In [ ]:
def easy_ndvi(redfile, nirfile, outfile):
  '''
  Calculates NDVI from a red and NIR raster band file and saves it as a Geotiff raster.

  Args:
    redfile = string of the filename and directory path to the raster file containing the red band
    nirfile = string of the filename and directory path to the raster file containing the NIR band
    outfile = string of the output filename and directory path for the NDVI raster file
  '''

  # open the red band file
  redfile = rasterio.open(redfile, 'r')
  # load the data from the red band file
  band_red = redfile.read(1)

  # open the NIR band file
  nirfile = rasterio.open(nirfile, 'r')
  # load the data from the NIR band file
  band_nir = nirfile.read(1)

  # The Sentinel-2 bands are delivered as uint16 data type (unsigned integer 16 bits per pixel).
  # This means that we cannot do floating point calculations on them without first converting them to float.
  # Convert the band arrays to float:
  band_red = np.float32(band_red)
  band_nir = np.float32(band_nir)

  # Calculate the vegetation index. This is done pixel by pixel using the NumPy masked array arithmetic.

  # We need to handle exceptions to the calculation. Where the sum of the two bands
  # in the denominator is zero (NIR+Red), the NDVI formula would give an error otherwise.
  # We do this by setting the NumPy error state to 'ignore' for this calculation only:
  # https://numpy.org/doc/stable/reference/generated/numpy.errstate.html

  with np.errstate(divide='ignore'): # this only applies to the following indented lines of code
    # NDVI formula:
    ndvi = np.divide((band_nir - band_red), (band_nir + band_red)) # ignore division by zero errors here
    ndvi[(band_nir + band_red) == 0] = 0 # where NIR + Red is zero, set the NDVI to zero

  # save the NDVI image
  outfile = rasterio.open(outfile, 'w', driver="GTiff", width=redfile.width,
                          height=redfile.height, count=1, crs=redfile.crs,
                          transform=redfile.transform, dtype=np.float32)
  outfile.write(ndvi, 1)
  outfile.close()

  return()

In [ ]:
def easy_plot(afile, ax, bands=[3,2,1], percentiles=[0,100], cmap="binary_r",
              xlim=None, ylim=None, shapefile=None, fillcolor=None, linecolor=None,
              title=None, fontsize=None):
  '''
  Visualise a raster image file in GeoTIFF format on a specified axis of a figure in matplotlib.
  Red, Green and Blue bands for the visualisation can be specified.
  Optionally, percentiles for cropping extreme values can be specified.
  Also optional is the definition of x and y limits of coordinates for zooming
  into the raster image.
  The raster will be converted to uint8 data type for plotting.
  Optionally, a shapefile can be plotted on top of the raster.

  Args:
    afile = string with directory path and filename of the raster file
    ax = matplotlib axes object on which the image will be plotted
    bands (optional) = list of three integer values specifying which bands of the
      raster will be displayed as red, green and blue (default = [3,2,1])
    percentiles (optional) = list of two values from 0 to 100 specifying the extreme
      values that will be excluded from the visualisation. Default is that all
      values will be plotted = [0,100]
    cmap (only used for single band plots) = matplotlib colormap, defaults to reverse binary (black to white)
    xlim (optional) = list of two x coordinate values in map units for zooming into the raster
    ylim (optional) = list of two y coordinate values in map units for zooming into the raster
    shapefile (optional) = string with directory path and filename of the shapefile
    fillcolor (optional) = fill colour of the polygons from the shapefile
    linecolor (optional) = line colour of the polygons from the shapefile
    title (optional) = string with a plot title
    fontsize (optional) = fontsize for the plot title
  '''

  def scale_to_uint8(x, percentiles=[0,100]):
    '''
    Scales an array to the range from 0-255 and converts the data type to uint8.
    NaN values will be ignored.

    Args:
      x = input array
      percentiles = list of length 2 of percentiles for trimming the histogram (0-100)

    Returns:
      Scaled array of uint8 data type
    '''

    x = np.float32(x)
    amin = np.nanpercentile(x, percentiles[0])
    amax = np.nanpercentile(x, percentiles[1])
    anewmin = 0.0
    anewmax = 255.0

    # apply percentile thresholds
    x[x<amin] = amin
    x[x>amax] = amax

    # scale values
    xscaled = (x - amin) / (amax - amin) * (anewmax - anewmin) + anewmin

    return(xscaled.astype(np.uint8))

  # open the input raster
  with rasterio.open(afile, 'r') as src:

    # save the uint8 image as a temporary Geotiff file with only 3 bands
    with rasterio.open('tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff',
                        'w', driver='Gtiff', width=src.width, height=src.height,
                        count=3, crs=src.crs, transform=src.transform,
                        dtype=np.uint8) as tmpfile:

      # mask out extreme values for each band
      for b in range(len(bands)):
        # read band data
        a = src.read(bands[b])
        a_uint8 = scale_to_uint8(a, percentiles)
        # write the output into the new file as band b+1
        tmpfile.write(a_uint8, b+1)

    # close the files
    tmpfile.close()
    src.close()

    # plot the raster
    with rasterio.open(r'tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff', 'r') as imgfile:

      if (xlim==None):
        xlim=[imgfile.bounds.left, imgfile.bounds.right]
        # afile.bounds returns a BoundingBox(left, bottom, right, top) object,
        #    from which we need to get the corner coordinates like so

      if (ylim==None):
        ylim=[imgfile.bounds.bottom, imgfile.bounds.top]

      # zoom in to an area of interest by setting the axes limits of our map
      ax.set_xlim(xlim)
      ax.set_ylim(ylim)
      if len(bands) == 1:
        rasterio.plot.show(imgfile.read(bands[0]), ax=ax, cmap=cmap, transform=imgfile.transform)
      else:
        rasterio.plot.show(imgfile, ax=ax, transform=imgfile.transform)

      # close the temporary file
      imgfile.close()

    # and remove the temporary file when we do not need it anymore
    os.remove('tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff')

  # Use the Geopandas library for plotting the shapefile on top of the raster image,
  #   if a shapefile name is defined
  if shapefile is not None:
    shp = gpd.read_file(shapefile)

    if fillcolor is None:
      fillcolor = "none"

    if linecolor is None:
      linecolor = "yellow"

    shp.plot(ax=ax, facecolor=fillcolor, edgecolor=linecolor)

  # add an optional title for the plot if one is given
  if fontsize is None:
    fontsize = 8

  ax.set_title(title, fontsize=fontsize)

In [ ]:
def sample_raster(afile, band=1, n=1000, missing=65535):
  '''
  Draws a random sample of n pixel values from a raster but excludes missing values.
  Sampling is done without replacement.
  Returns a list of pixel values for a single band.
  Pixel locations are recorded in the function but are not currently returned.

  Args:
    afile = file name of the raster file
    band = number of the single band to be sampled
    n = number of samples
    missing = No Data value in the raster
  '''

  random.seed(1)

  pxs = [] # list of pixel coordinates in x
  pys = [] # list of pixel coordinates in y
  vals = [] # list of pixel values

  dataset = rasterio.open(afile, 'r')

  while True:

    # generate a random pixel coordinate pair
    px = int(random.random()*dataset.width)
    py = int(random.random()*dataset.height)

    # create 1x1px window of the pixel
    window = rasterio.windows.Window(px - 1//2, py - 1//2, 1, 1)

    # read rgb values of the single-pixel window but exclude missing values
    clip = dataset.read(band, window=window)

    if clip[0][0] != missing:
      if px not in pxs and py not in pys:
        vals.append(clip[0][0])
        pxs.append(px)
        pxs.append(py)

    if len(vals)==n:
      break

  return(vals)

In [ ]:
def hist_sample(imagefile, n=1000, missing=65535, title="histogram", **kwargs):
  '''
  shows a histogram of a pixel sample from a raster image file with one or more bands

  Args:
    imagefile = path and file name of the raster image
    n (optional) = sample size
    missing (optional) = missing value to be excluded from the histogram plot
    title (optional) = plot title
    kwargs (optional) = optional parameters for Matplotlib

  Returns a figure and axes object
  '''

  # unpack the keyword arguments for the figure object
  fig_kwargs = {"figsize" : kwargs.get("figsize", (6,6)),
                "dpi" : kwargs.get("dpi", None),
                "facecolor" : kwargs.get("facecolorfig", "white"),
                "edgecolor" : kwargs.get("edgecolor", "black"),
                "linewidth" : kwargs.get("linewidth", 1),
                "frameon" : kwargs.get("frameon", None),
                "subplotpars" : kwargs.get("subplotpars", None),
                "tight_layout" : kwargs.get("tight_layout", None),
                "constrained_layout" : kwargs.get("constrained_layout", None)
                }

  # unpack the keyword arguments for the axes.hist object
  hist_kwargs = {"bins" : kwargs.get("bins", 20),
                 "range" : kwargs.get("range", None),
                 "density" : kwargs.get("density", None),
                 "weights" : kwargs.get("weights", None),
                 "cumulative" : kwargs.get("cumulative", False),
                 "bottom" : kwargs.get("bottom", None),
                 "histtype" : kwargs.get("histtype", "bar"),
                 "align" : kwargs.get("align", "mid"),
                 "orientation" : kwargs.get("orientation", "vertical"),
                 "log" : kwargs.get("log", False),
                 "color" : kwargs.get("color", "skyblue"),
                 "label" : kwargs.get("label", None),
                 "stacked" : kwargs.get("stacked", False),
                 "data" : kwargs.get("data", None),
                 "linewidth" : kwargs.get("linewidth", 1),
                 "antialiased" : kwargs.get("antialiased", None),
                 "hatch" : kwargs.get("hatch", None),
                 "fill" : kwargs.get("fill", True),
                 "capstyle" : kwargs.get("capstyle", "round"),
                 "joinstyle" : kwargs.get("joinstyle", "miter"),
                 "agg_filter" : kwargs.get("agg_filter", None),
                 "alpha" : kwargs.get("alpha", None),
                 "clip_box" : kwargs.get("clip_box", None),
                 "clip_on" : kwargs.get("clip_on", None),
                 "clip_path" : kwargs.get("clip_path", None),
                 "gid" : kwargs.get("gid", None),
                 "edgecolor" : kwargs.get("edgecolor", None),
                 "facecolor" : kwargs.get("facecolorhist", None),
                 "in_layout" : kwargs.get("in_layout", False),
                 "linestyle" : kwargs.get("linestyle", "-"),
                 "rasterized" : kwargs.get("rasterized", None),
                 "snap" : kwargs.get("snap", None),
                 "url" : kwargs.get("url", None),
                 "visible" : kwargs.get("visible", True),
                 "zorder" : kwargs.get("zorder", None)
                  }

  # get number of bands
  ds = rasterio.open(imagefile, 'r')
  bands = ds.count
  ds = None

  # create a figure and axes object
  fig, ax = plt.subplots(1, bands, **fig_kwargs)

  for b in range(bands):
    print(b)
    # sample the raster bands
    pixels = sample_raster(imagefile, b+1, n, missing)

    # plot the histograms
    t = title + '_' + str(b+1)
    ax[b].hist(pixels, **hist_kwargs)

    # add a title if specified
    if t is not None:
      ax[b].set_title(t)

  return(fig, ax)

In [ ]:
def training_shapefile_to_raster(training_shapefilename, inraster_filename, outraster_filename, verbose=False):
  '''
  Reads in a shapefile with training polygons and produces a raster file that
    aligns with an input rasterfile (same corner coordinates, resolution, coordinate
    reference system and geotransform). Each pixel value in the output raster will
    indicate the class number of the training shapefile based on the attribute column
    named 'Class' with a capital C.

  Based on https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python

  Args:
    training_shapefilename = string pointing to the input training shapefile in ESRI format
    inraster_filename = string pointing to the input raster file that we want to align the output raster to
    outraster_filename = string pointing to the output raster file
    verbose (optional)= True or False. If True, additional text output will be printed.
  '''

  # Open the shapefile with GeoPANDAS
  shp = gpd.read_file(training_shapefilename)

  # Open the input raster file with RasterIO
  inraster = rasterio.open(inraster_filename, 'r')
  # Reproject the geometries from the shapefile to the CRS of the raster
  shp = shp.to_crs(inraster.crs)

  # copy and update the metadata from the input raster for the output
  meta = inraster.meta.copy()
  meta.update(dtype=np.uint8)
  meta.update(count=1)

  # Now burn the features into the raster and write it
  with rasterio.open(outraster_filename, 'w', **meta) as outraster:
    # create the output array as a Numpy array filled with zeros and of the same
    #    shape as the input raster
    raster_shape = inraster.read(1).shape
    out_arr = np.zeros(raster_shape, dtype=np.uint8)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom, value) for geom, value in zip(shp.geometry, shp.Class))
    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr,
                                transform=outraster.transform, all_touched=False)

    if verbose:
      print("Shapefile attribute table and polygon geometries:")
      pprint(shp)

      print("shapes is a generator object:")
      print(shapes)

      print("class values and geometries:")
      for geom, value in zip(shp.geometry, shp.Class):
        print(value, geom)

      print("Rasterised layer:")
      print(burned)

      print("The output raster has values from ", np.min(burned), " to ", np.max(burned))

    outraster.write(burned.astype(np.uint8), 1)
    outraster = None

  inraster = None

In [ ]:
def train_rf_model(raster, samples, modelfile, ntrees = 101, weights = None):
    '''
    Trains a random forest classifier model based on a raster file with bands
      as features and a second raster file with training data, in which pixel
      values indicate the class.

    Args:
      raster = filename and path to the raster file to be classified in tiff format
      samples = filename and path to the raster file with the training samples
        as pixel values (in tiff format)
      modelfile = filename and path to a pickle file to save the trained model to
      ntrees (optional) = number of trees in the random forest, default = 101
      weights (optional) = a list of integers giving weights for all classes.
        If not specified, all weights will be equal.

    Returns:
      random forest model object
    '''

    # read in raster from geotiff
    img_ds = io.imread(raster)

    # convert to 16bit numpy array
    img = np.array(img_ds, dtype='int16')

    # read in the training sample pixels
    roi_ds = io.imread(samples)
    roi = np.array(roi_ds, dtype='int8')

    # read in the class labels
    labels = np.unique(roi[roi > 0])
    nclasses = labels.size # number of unique class values
    print('The training data include {n} classes: {classes}'.format(n=nclasses, classes=labels))

    # compose the X,Y pixel positions (feature dataset and training dataset)
    # 0 = missing class value
    X = img[roi > 0, :]
    Y = roi[roi > 0]

    # create a dictionary of class weights (class 1 has the weight 1, etc.)
    w = dict() # create an empty dictionary
    for i in range(nclasses): # iterate over all classes from 0 to nclasses-1
      if weights == None:
        w[i+1] = '1' # if not specified, set all weights to 1
      else:
        if weights.size >= nclasses: # if enough weights are given, assign them
          w[i+1] = weights[i] # assign the weights if specified by the user
        else: # if fewer weights are defined than the number of classes, then set the remaining weights to 1
          if i > weights.size:
            w[i+1] = '1' # set weight to 1
          else:
            w[i+1] = weights[i] # assign the weights if specified by the user

    # build the Random Forest Classifier
    # for more information: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

    rf = RandomForestClassifier(class_weight = weights, n_estimators = ntrees, criterion = 'gini', max_depth = 4,
                                min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto',
                                bootstrap = True, oob_score = True, n_jobs = 1, random_state = None, verbose = True)

    # fit the model to the training data and the feature dataset
    rf = rf.fit(X,Y)

    # export the Random Forest model to a file
    joblib.dump(rf, modelfile)

    # calculate the feature importances
    importances = rf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices], color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()

    # Out-of-bag error rate as a function of number of trees:
    oob_error = [] # define an empty list with pairs of values

    # Range of `n_estimators` values to explore.
    mintrees = 30 # this needs to be a sensible minimum number to get reliable OOB error estimates
    maxtrees = max(mintrees, ntrees) # go all the way to the highest number of trees
    nsteps = 5 # number of steps to calculate OOB error rate for (saves time)

    # work out the error rate for each number of trees in the random forest
    for i in range(mintrees, maxtrees + 1, round((maxtrees - mintrees)/nsteps)): # start, end, step
      rf.set_params(n_estimators=i)
      rf.fit(X, Y)
      oob_error.append((i, 1 - rf.oob_score_))

    # Plot OOB error rate vs. number of trees
    xs, ys = zip(*oob_error)
    plt.plot(xs, ys)
    # plt.xlim(0, maxtrees)
    plt.xlabel("n_estimators")
    plt.ylabel("OOB error rate")
    # plt.legend(loc="upper right")
    plt.show()

    return(rf) # returns the random forest model object

In [ ]:
def classify_rf(raster, modelfile, outfile, verbose = False):
  '''
  Reads in a pickle file of a random forest model and a raster file with feature layers,
    and classifies the raster file using the model.

  Args:
    raster = filename and path to the raster file to be classified (in tiff uint16 format)
    modelfile = filename and path to the pickled file with the random forest model in uint8 format
    outfile = filename and path to the output file with the classified map in uint8 format
    verbose (optional) = True or False. If True, provides additional printed output.
  '''

  # Read Data
  src = rasterio.open(raster, 'r')
  img = src.read()

  if verbose:
    print("img.shape = ", img.shape)

  # get number of bands
  n = img.shape[0]

  if verbose:
    print(n, " Bands")

  # load your random forest model from the pickle file
  clf = joblib.load(modelfile)

  # to work with SciKitLearn, we have to reshape the raster as an image
  # this will change the shape from (bands, rows, columns) to (rows, columns, bands)
  img = reshape_as_image(img)

  # next, we have to reshape the image again into (rows * columns, bands)
  # because that is what SciKitLearn asks for
  new_shape = (img.shape[0] * img.shape[1], img.shape[2])

  if verbose:
    print("img[:, :, :n].shape = ", img[:, :, :n].shape)
    print("new_shape = ", new_shape)

  img_as_array = img[:, :, :n].reshape(new_shape)

  if verbose:
    print("img_as_array.shape = ", img_as_array.shape)

  # classify it
  class_prediction = clf.predict(img_as_array)

  # and reshape the flattened array back to its original dimensions
  if verbose:
    print("class_prediction.shape = ", class_prediction.shape)
    print("img[:, :, 0].shape = ", img[:, :, 0].shape)

  class_prediction = np.uint8(class_prediction.reshape(img[:, :, 0].shape))

  if verbose:
    print(class_prediction.dtype)

  # save the image as a uint8 Geotiff file
  tmpfile = rasterio.open(outfile, 'w', driver='Gtiff',
                          width=src.width, height=src.height,
                          count=1, crs=src.crs, transform=src.transform,
                          dtype=np.uint8)

  tmpfile.write(class_prediction, 1)

  tmpfile.close()

In [ ]:
def easy_plot_landcovermap(classfile, ax, shapefile=None, band=1, xlim=None, ylim=None,
                           labels=None, colours=None, verbose=False):
  '''
  Visualise a raster image file on a specified axis of a figure in matplotlib.
  The specified band must contain class values in uint8 data type.
  Optionally, x and y limits of a coordinate window for zooming in can be given.
  Inspired by https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/classify-plot-raster-data-in-python/

  Args:
    afile = string with directory path and filename of the raster file
    ax = matplotlib axes object on which the image will be plotted
    shapefile (optional) = string with directory path and filename of a shapefile to be plotted on top of the raster
    band (optional) = number of the band containing the classes, default = 1
    xlim (optional) = list of two x coordinate values in map units for zooming into the raster
    ylim (optional) = list of two y coordinate values in map units for zooming into the raster
    labels (optional) = list of class labels of the land cover classes in string format
    colours (optional) = list of colours for the land cover classes. See this chart
      for information on available colours: https://matplotlib.org/2.0.0/examples/color/named_colors.html
      Example:
        colours = ['mediumblue', 'firebrick', 'red', 'yellowgreen', 'gold', 'saddlebrown',
          'darkolivegreen']
    verbose (optional) = True or False. If True, additional printed output will be produced
  '''

  # open the rasterfile
  imgfile = rasterio.open(classfile, 'r')

  # read the band with the class values
  img = np.array(imgfile.read(band), dtype=np.uint8)


  # fix labels and colours - one label and one colour for each class in the raster
  unq = np.unique(img[img > 0]) # all unique values, excluding zero

  if labels is None:
    labels = np.unique(img[img > 0])
    nclasses = len(labels) # number of unique class values
  else:
    if len(labels) < len(unq): # if not enough labels have been defined by the user
      for i in range(len(labels), len(unq)):
        labels.append("class"+str(unq[i]))
    nclasses = len(labels) # number of unique class values

  if colours is None:
    cmap = cm.get_cmap('viridis', nclasses) # make a colormap with one color per class
  else:
    cmap = cm.get_cmap('viridis', nclasses) # make a colormap with one color per class
    if len(colours) < nclasses: # if not enough colours have been defined by the user
      additional_colours = plt.cm.get_cmap('viridis', nclasses-colours.size)
        # Map each index in 0, 1, ..., n-1 to a distinct RGB color.
        # The keyword argument name must be a standard mpl colormap name
      colours.append(additional_colours)

  print('The training data include {n} classes: {classes}'.format(n=nclasses, classes=labels))

  # calculate the bin boundaries between class values, e.g. 0.5-1.5 for class 1
  class_bins = [i+0.5 for i in range(nclasses+1)]

  # Generate a colourmap index based on discrete intervals
  norm = matplotlib.colors.BoundaryNorm(class_bins, nclasses)

  # set a title for the plot
  mytitle = "Land cover map"

  # zoom in if xlim and ylim are specified
  if (xlim==None):
    xlim=[imgfile.bounds.left, imgfile.bounds.right]
    # afile.bounds returns a BoundingBox(left, bottom, right, top) object,
    #    from which we need to get the corner coordinates like so
  if (ylim==None):
    ylim=[imgfile.bounds.bottom, imgfile.bounds.top]
  ax.set_xlim(xlim)
  ax.set_ylim(ylim)

  # Plot the classified land cover raster
  rasterio.plot.show(imgfile, ax=ax, cmap=cmap, norm=norm, title=mytitle)
  imgfile.close()

  if shapefile is not None:
    # use the Geopandas library for plotting the shapefile
    shp = gpd.read_file(shapefile)
    shp.plot(ax=ax, facecolor="none", edgecolor="black")

In [ ]:
def easy_zonal_stats(rasterfiles, shapefile, statisticsfile, nodata=0,
                     stats="count min mean max median"):
  '''
  Extracts zonal statistics using rasterstats from a list of raster files and saves
    a single statisticsfile in .csv format as output.

  Args:
    rasterfiles = list of strings with raster file names with full direcroy paths
    shapefile = string giving the path and file name of the shapefile
    statisticsfile = string with the name of the output .csv file
    nodata (optional) = nodata value to be ignored in the calculation
    stats (optional) = string defining which statistics are calculated

  Returns: a Pandas dataframe with the statistics results saved into the statisticsfile
  '''

  # iterate over all raster files and extract zonal statistics
  for x in range(len(rasterfiles)):
    f = rasterfiles[x]
    # extract the string between the last slash and the dot in the file name and directory path as scene ID
    scene_id = f.split("/")[-1].split(".")[0]

    # get zonal statistics for all polygons in the shapefile
    # the result is a list of dictionaries
    statistics = zonal_stats(shapefile, f, nodata=nodata, stats=stats)

    # get number of rows for the output file as the product of n * m
    n = len(statistics) # number of polygons
    m = len(rasterfiles) # number of files

    if x == 0:
      # create the pandas dataframe in the first iteration with column names only
      df = pd.DataFrame(columns=['scene_id'].append(stats.split(" ")))

    # add the scene ID to each row in the statistics output from this scene (image)
    for row in range(n):
      statistics[row].update({"scene_id": scene_id})

    # append rows to the dataframe in each iteration with the scene_id as index
    for s in statistics: # iterate over the list of dictionaries (one for each polygon)
      df = df.append(s, ignore_index=True)

  # After the loop, write the statistics dataframe to a .csv file (overwrite if exists)
  df.to_csv(statisticsfile, index=False)

  return(df)

In [ ]:
def remove_radiometric_offset_from_N0400(in_file,
                                         BOA_ADD_OFFSET = 1000):
    """
    This function is losely based on a function written by Dr Ivan Reading, 2022-06-07
    It fixes a data inconsistency in the Sentinel-2 time series introduced in March 2022:
    Since processing baseline 04.00, there is a radiometric constant added to each band.
    This function reads in an image in a recognised format and removes that constant.
    More information can be found at:
    https://forum.step.esa.int/t/info-introduction-of-additional-radiometric-offset-in-pb04-00-products/35431

    Parameters
    ----------
    in_file : str
        Path to the input raster file. It must be in a Sentinel-2 SAFE file directory
    BOA_ADD_OFFSET : int
        Required offset per band (from xml information within L2A SAFE file directory)

    Returns
    -------
    out_file : str
        The path to the output image file

    """
    def get_processing_baseline(file_path):
      # works if safe_path contains the full SAFE directory name and
      #   points to a .jp2 file with the band data

      # get baseline from the SAFE file directory name
      safe_id = file_path.split("/")[-6]
      baseline = safe_id.split("_")[3].split(".")[0]

      # if a baseline is included in the image file name, use that instead
      # this will start with the letter A and indicate that the file has
      # already been offset corrected
      file_name_start = file_path.split("/")[-1]
      file_baseline = file_name_start.split("_")[-1].split(".")[0]
      if file_baseline.startswith('A'):
        return file_baseline
      else:
        return baseline

    def set_processing_baseline(file_path, new_baseline):
      # Appends a letter 'A' and the processing baseline to the file name
      file_path_dir = os.path.split(file_path)[0]
      file_path_start = os.path.split(file_path)[1].split(".")[0]
      file_path_end = os.path.split(file_path)[1].split(".")[1]
      new_file_path = os.path.join(file_path_dir,
                                   file_path_start + "_" + new_baseline + "." + file_path_end)
      return new_file_path

    def add_number_to_band(x, offset):
      '''
      Adds an offset value to each pixel in an array.

      Args:
        x = input array
        offset = a number that will be added to each pixel value in x.
                 Must be the same data type as the array x

      Returns:
        array (if the function runs OK)
        -1 as an error code if the data types do not match
      '''

      #if x.dtype != np.dtype(offset):
      #  print("ERROR: Offset must be of same data type as array.")
      #  return(-1)
      #else:
      x = np.float32(x)
      x1 = x + offset
      return(x1)

    # Check out_file exists and report an error if not
    print("Removing radiometric offset from processing baseline N0400 onwards.")
    if not os.path.exists(in_file):
      print("ERROR: File does not exist: "+in_file)
      return -1
    else:
      #TODO: Read individual BOA_ADD_OFFSET value for each band from xml information in SAFE file root
      # get the file name and remove the directory path
      f = os.path.split(in_file)[1]
      # detect file driver of input file
      driver = rasterio.drivers.driver_from_extension(f)
      # extract string of processing baseline from file name
      baseline = get_processing_baseline(in_file)[1:]
      if get_processing_baseline(in_file)[0] == 'A':
        print(f'Offset already applied - file marked as: {get_processing_baseline(in_file)}')
        return(in_file)
      else:
        if int(baseline) >= 400:
          print(f'Removing radiometric offset from file: {f}')
          print(f'Processing baseline: {baseline}')
          # create a name for the output file
          out_file = set_processing_baseline(in_file, f'A{baseline:s}')
          print(f'Output file: {out_file}')
          # open the input raster file
          with rasterio.open(in_file, 'r') as src:
            # get number of bands in the input raster file
            bands = src.count
            # create the output file
            with rasterio.open(out_file, 'w', driver=driver, width=src.width,
                              height=src.height, count=bands, crs=src.crs,
                              transform=src.transform, dtype=src.dtypes[0]) as dst:
              for band in range(bands):
                # read band data
                a = src.read(band+1)
                # add the offset
                a1 = add_number_to_band(a, BOA_ADD_OFFSET)
                # write the band to the output file
                dst.write(a1, band+1)
              # close the files
              dst.close()
              src.close()
              return out_file
        else:
          print('Offset does not need to be applied. Processing baseline is below 04.00')
          return in_file